In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from sentence_transformers import SentenceTransformer, util
import torch

# ---------- 2. Load models ----------
# Model for semantic similarity
sim_model = SentenceTransformer("microsoft/codebert-base")

# Commit predictor model (for rectification)
pred_model_name = "mamiksik/CommitPredictorT5"
pred_tokenizer = AutoTokenizer.from_pretrained(pred_model_name)
pred_model = AutoModelForSeq2SeqLM.from_pretrained(pred_model_name)

No sentence-transformers model found with name microsoft/codebert-base. Creating a new one with mean pooling.
c:\Users\Surrya Gokul\anaconda3\envs\genai\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Surrya Gokul\.cache\huggingface\hub\models--microsoft--codebert-base. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


In [ ]:
from tqdm import tqdm
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
pred_model = pred_model.to(device)

def batch_rectify_messages(diffs, batch_size=16):
    cleaned = [clean_diff(d) for d in diffs]
    results = []

    # tqdm gives ETA automatically
    for i in tqdm(range(0, len(cleaned), batch_size), desc="Generating commits"):
        batch = cleaned[i:i+batch_size]
        inputs = pred_tokenizer(
            batch,
            return_tensors="pt",
            truncation=True,
            max_length=512,
            padding=True
        ).to(device)

        with torch.no_grad():
            outputs = pred_model.generate(**inputs, max_length=64)

        decoded = pred_tokenizer.batch_decode(outputs, skip_special_tokens=True)
        results.extend(decoded)

    return results

df["LLM Inference"] = batch_rectify_messages(df["Diff"].tolist(), batch_size=16)


Generating commits: 100%|██████████| 979/979 [1:51:09<00:00,  6.81s/it]  


In [ ]:
df.to_csv("llm_inference.csv", index=False)